In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from tools import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from scipy.stats import multivariate_normal
from scipy.misc import derivative
from scipy.linalg import fractional_matrix_power
from scipy.stats import wishart
from IPython.display import Latex
import sys
import time

In [3]:
def Eff_quantile (Y,p):
    rho0=0; n = len(Y); 
#     quantile = scipy.stats.norm.ppf(percent,loc=mu,scale=std)
    quantile = np.quantile(Y,p)
    x = np.where(Y<=quantile,1,0);Tint=np.var(x);
    for irho in range(1,(n-10)):
        rho = np.cov(x[0:n-irho], x[irho:n])[0,1]
        if ((irho>10) & (rho+rho0<0)):
            break;
        Tint += rho*2;
        rho0 = rho;
#         print('lag:',irho,'|',rho,'sum',':')
    return (p*(1-p)/Tint);

In [4]:
def logtargetpdf(x, target, mu=0, sigma=1):
    lnp = 0.0
    if (target == 'Exp'):
            lnp = -(x-mu)/sigma if (x>=mu) else -500
    elif (target == 'Folded_Gaussian'):
            lnp = -(x-mu)**2/(2*sigma**2) if (x>=mu) else -500
    elif (target == 'N01_T2'):
            lnp = -(x-mu)**2/(2*sigma**2)
    elif (target == 'N01_Logistic'):
            lnp = -(x-mu)**2/(2*sigma**2)
    return (lnp)

def transform_x(x,transform,mu=0,sigma=1):
    y = 0.0
    x = (x-mu)/sigma
    if (transform == 'Exp(1)_CDF'):
        y = np.exp(-x)
    elif (transform == 't2_CDF'):
        y = 1/2+x/np.sqrt(8+4*x**2)
    elif (transform == 'logistic_CDF'):
        y = 1/(1+np.exp(-x))
    return y

def transform_inverse(y,transform,mu=0,sigma=1):
    x = 0.0
    if (transform == 'Exp(1)_CDF'):
        x = -np.log(y)
    elif (transform == 't2_CDF'):
        x = (2*y-1)/np.sqrt(2*y*(1-y))
    elif (transform == 'logistic_CDF'):
        x = -np.log(1/y-1)
    x = x * sigma + mu
    return x

def log_proposal_ratio(x,transform,burnin_off,kernel,mu=0,sigma=1):
    ratio = 0.0
    if ('Trans' in kernel)&burnin_off:
        x = (x-mu)/sigma
        if (transform == 'Exp(1)_CDF'):
            ratio = -x
        elif (transform == 't2_CDF'):
            ratio = -3/2*np.log(2+x**2) - np.log(sigma)
        elif (transform == 'logistic_CDF'):
            ratio = -x -np.log(sigma) - 2*np.log(1+np.exp(-x))
    return ratio

In [6]:
#MCMC sampling implementation
# np.random.seed(121)
begin_total = time.time()
nround = int(10e5)
burnin = int(2*10e4)
size = len(range(-burnin,nround))
sigmaset_baseline = np.array([0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 1, 1.1, 1.3, 1.5, 1.7,
                    1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.8,
                    3.0, 3.2, 3.5, 4, 4.5, 5, 6, 7, 8])
kernelset = ['Uniform','TransUniform','TransBactrian','TransStrawHat']
targetset = {'N01_T2':'t2_CDF', 'N01_Logistic':'logistic_CDF'}
# kernelset = ['Uniform']
# targetset = {'N01_T2':'t2_CDF'}

target_result = {}
mu=3;std=4
for target,transform in targetset.items():
    eff_kernel = {}; Pjump_kernel = {}; Exx2_kernel = {}; Rho_kernel={}; Sample_kernel={}; Quantile_kernel={}
    for kernel in kernelset:
        Eff = []; Pjump = []; Exx2=[]; Rho=[]; Sample=[]; Quantile=[]
        if 'Uniform' in kernel:
            random_result = np.random.uniform(low=-np.sqrt(3), high=np.sqrt(3), size=size)
        elif 'Bactrian' in kernel:
            random_result = random_bactrian(size = size)
        elif 'StrawHat' in kernel:
            random_result = random_strawhat(size=size)
        sigmaset = sigmaset_baseline if 'Trans' in kernel else sigmaset_baseline*std            
        for sigma in sigmaset:
            begin_loop = time.time()
            exx2 = 0; accept = 0; burn_in_result = []
            x = np.random.uniform(low=mu,high=mu+1)
            result = np.zeros(nround)
            logpdf_old = logtargetpdf(x,target,mu=mu,sigma=std) - log_proposal_ratio(x,transform,True,kernel,mu=mu,sigma=std)
            mu_hat, sigma_hat = mu,std
            burnin_off=False
            for j in range(-burnin,nround):
                if (j==0):
                    accept = 0; burnin_off = True
                    mu_hat, sigma_hat = np.mean(burn_in_result), np.std(burn_in_result)
                y = x if (not(('Trans' in kernel)&burnin_off)) else transform_x(x,transform,mu=mu_hat,sigma=sigma_hat)
                ynew = y + random_result[j+burnin] * sigma
                if ((('Trans' in kernel)&burnin_off) & (not (0 <= ynew <= 1))):
                    ynew = reflect(ynew, xL=0, xU=1)
                xnew = ynew if (not(('Trans' in kernel)&burnin_off)) else transform_inverse(ynew,transform,mu=mu_hat,sigma=sigma_hat)
                logpdf_new = logtargetpdf(xnew,target,mu=mu,sigma=std) - log_proposal_ratio(xnew,transform,burnin_off,kernel,mu=mu_hat,sigma=sigma_hat)
                pi_dif = logpdf_new - logpdf_old
                if ((pi_dif>0) | (np.exp(pi_dif)>np.random.random())):
                    x = xnew
                    logpdf_old = logpdf_new
                    accept += 1
                
                if (j<0):
                    burn_in_result.append(x)
                else:
                    result[j] = x
                    
            eff = Eff_IntegratedCorrelationTime(result)
            rho = [rho_k(result,i) for i in range(1,11)]
            exx2 = sum(np.square(result[1:] - result[:-1]))
            quantile = [Eff_quantile(result,i) for i in [0.05,0.25,0.5,0.75,0.95]]
            Eff.append(eff)
            Pjump.append(accept/nround)
            Exx2.append(exx2/nround)
            Rho.append(rho)
            Sample.append(result.copy())
            Quantile.append(quantile)
            display(Latex('Target:{} | Kernel:{} | $\sigma$:{} | eff:{} | $P_{{jump}}$:{} | Exx2:{} | $\\rho_1$:{} | $\\rho_2$:{} | $E_{{0.05}}$:{} | $E_{{0.25}}$:{} | $E_{{0.50}}$:{} | Time:{}'.format(
                    target,kernel,sigma,round(eff,4),round(accept/nround,4),round(exx2/nround,4),round(rho[0],4),round(rho[1],4),
                    round(quantile[0],4),round(quantile[1],4),round(quantile[2],4),round((time.time()-begin_loop),2))))
        eff_kernel[kernel] = Eff
        Pjump_kernel[kernel] = Pjump
        Exx2_kernel[kernel] = Exx2
        Rho_kernel[kernel] = Rho
        Sample_kernel[kernel] = Sample
        Quantile_kernel[kernel] = Quantile
    target_result[target] = [eff_kernel,Pjump_kernel,Exx2_kernel,Rho_kernel,Sample_kernel,Quantile_kernel]
print('总共用时: %f'% (time.time()-begin_total))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

总共用时: 3695.462614


In [7]:
name = ['Kernel' ,'Sigma', 'Eff', 'Pjump', 'Exx2', 'rho1', 'rho2', 'E_5%', 'E_25%', 'E_50%']
print('{:<25} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}'.format(*name))

for target in targetset:
    kernel_info = {}
    print('                                      ',target)
    for kernel in kernelset:
        index = [np.argmax(target_result[target][0][i]) for i in kernelset]
        index_need = index[np.where(np.array(kernelset)==kernel)[0][0]]
        info = [target_result[target][i][kernel][index_need] for i in range(len(target_result[target]))]
        rho1 = info[3][0]; rho2 = info[3][1]
        E05 = info[5][0]; E25 = info[5][1]; E50 = info[5][2]
#         kernel_info[kernel] = np.round(np.concatenate([[sigmaset[index_need]],info[0:3],[rho1,rho2]]),3)
        kernel_info[kernel] = np.round(np.concatenate([[sigmaset[index_need]],info[0:3],[rho1,rho2],[E05,E25,E50]]),3)
        kernel_info[kernel] = kernel_info[kernel].tolist()
        kernel_info[kernel].insert(0,kernel)
        print('{:<25} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}'.format(*kernel_info[kernel]))

Kernel                    Sigma    Eff      Pjump    Exx2     rho1     rho2     E_5%     E_25%    E_50%   
                                       N01_T2
Uniform                   2.2      0.281    0.408    14.148   0.559    0.315    0.298    0.253    0.244   
TransUniform              0.9      0.975    0.832    31.575   0.014    0.0      0.79     0.838    0.847   
TransBactrian             0.9      1.573    0.832    39.409   -0.227   0.057    0.759    0.958    1.329   
TransStrawHat             1.0      1.73     0.838    41.016   -0.278   0.088    0.711    0.873    1.552   
                                       N01_Logistic
Uniform                   2.2      0.277    0.408    14.053   0.559    0.317    0.294    0.256    0.237   
TransUniform              0.8      0.818    0.717    29.366   0.086    0.014    0.666    0.619    0.623   
TransBactrian             1.0      1.316    0.722    36.504   -0.143   0.024    0.685    0.731    0.841   
TransStrawHat             0.9      1.536    0.